# Generating a Data Set of Optimal Trajectories

In [ ]:
# We first import the resources
import sys
sys.path.append('..')
from Trajectory import Point_Lander
from Optimisation import Hermite_Simpson
from PyGMO import *
from numpy import *
import matplotlib.pyplot as plt

In [ ]:
# We instantiate the problem
prob = Hermite_Simpson(Point_Lander(si=[0,1000,20,-5,9900]), nsegs=40)

In [ ]:
# Let us first supply a ballistic trajectory as a guess
zguess = prob.Guess.Ballistic(tf=28)

In [ ]:
# Decode the guess so we can visualise
tf, cb, s, c = prob.Decode(zguess)

In [ ]:
plt.plot(s[:,0], s[:,1], 'k.-')
plt.axes().set_aspect('equal', 'datalim')
plt.xlabel('Cross Range [m]')
plt.ylabel('Altitude [m]')
plt.show()

In [ ]:
# Use SLSQP and alternatively Monotonic Basin Hopping
alg1 = algorithm.scipy_slsqp(max_iter=3000, screen_output=True)
alg2 = algorithm.mbh(alg1, stop=1, screen_output=True)
alg3 = algorithm.ms(alg1)

In [ ]:
# Instantiate a population with 1 individual
pop = population(prob)
# and add the guess
pop.push_back(zguess)

In [ ]:
# We then optimise the trajectory
pop = alg2.evolve(pop)

In [ ]:
# We then save the resulting decision vector
z = pop.champion.x
save('HSD40', z)

In [ ]:
# We now visualise the optimised trajectory
z = load('HSD40.npy')
tf, cb, s, c = prob.Decode(z)
plt.plot(s[:,0], s[:,1], 'k.-')
plt.axes().set_aspect('equal', 'datalim')
plt.xlabel('Cross Range [m]')
plt.ylabel('Altitude [m]')
plt.show()

In [ ]:
# and look at the control throttle
plt.close('all')
f, ax = plt.subplots(2, sharex=True)
ax[0].plot(c[:,0], 'k.-')
ax[1].plot(cb[:,0], 'k.-')
plt.xlabel('Node Index')
ax[0].set_ylabel('Node Throttle')
ax[1].set_ylabel('Midpoint Throttle')
plt.show()

# Iterative Pertubation
We will now generate a database of optimal trajectories starting from an arbitrarily defined state threshold.

In [ ]:
# Defining the starting area
silb  = array([-100, 500, -100, -30, 8000], float)
siub  = array([100, 1000, 100, 10, 9000], float)
ntraj = 50 # Number of trajectories to generate
# We load the first trajectory we started with
z = load('HSD40.npy')

In [ ]:
# Then we iteratively optimise various trajectories
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.close('all')
plt.figure()
for i in range(ntraj):
    # Initialise the initial state randomly
    si = random.random()*(siub - silb) + silb
    prob = Hermite_Simpson(Point_Lander(si), nsegs=40)
    # create a population for the new problem
    pop = population(prob)
    # and add the previous population decision vector
    pop.push_back(z)
    # then evolve the trajectory
    pop = alg1.evolve(pop)
    # name the file to save to
    filename = 'Hermite_Simpson_Decision_' + str(i)
    # extract the decision vector
    z = pop.champion.x
    # save it!
    save(filename, z)
    # figure out what the decision means
    tf, cb, s, c = prob.Decode(z)
    # plot the trajectory
    plt.plot(s[:,0], s[:,1])
plt.xlabel('Cross Range [m]')
plt.ylabel('Altitude [m]')
plt.axes().set_aspect('equal', 'datalim')

plt.show()

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.close('all')
plt.figure()
fpre = 'Hermite_Simpson_Decision_'
for i in range(50):
    fname = fpre + str(i) + '.npy'
    z = load(fname)
    tf, cb, s, c = prob.Decode(z)
    plt.plot(s[:,0], s[:,1], 'k')
plt.title('Homotopic Trajectory Transitioning')
plt.xlabel('Cross Range [m]')
plt.ylabel('Altitude [m]')
plt.savefig('Homotopic_Data.png', format='png',transparent=True, bbox_inches='tight')
plt.show()